# Collecting Data From NBA API

In [61]:
# Import Libraries
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
# Get Player IDs of all Players

all_players = players.get_active_players()

print(all_players)

In [ ]:
# Get the career avg data of all players

df = pd.DataFrame()
iteration = 1

for player in all_players:
    try:
        print(f'Iteration: {iteration}')
        player_id = player.get('id', None)

        time.sleep(2)
    
        if player_id is not None:
            player_object = playercareerstats.PlayerCareerStats(per_mode36= 'PerGame', player_id= str(player_id))
            df_player = player_object.get_data_frames()[0]
            df = pd.concat([df, df_player])
    
    except KeyError as e: print(f'Skipping Player ID: {player_id}')
    except Exception as e: print(f'Trouble getting player {player_id}')

    iteration += 1
 
df
df.to_csv('../Datasets/nba.csv')

In [ ]:
# Get the career total data of all players

df_totals = pd.DataFrame()
failed_totals = []
iteration = 1

for player in all_players:
    try:
        print(f'Iteration: {iteration}')
        player_id = player.get('id', None)

        time.sleep(2)
    
        if player_id is not None:
            player_object = playercareerstats.PlayerCareerStats(per_mode36= 'Totals', player_id= str(player_id))
            df_player = player_object.get_data_frames()[0]
            df_totals = pd.concat([df_totals, df_player])
    
    except KeyError as e: 
        print(f'Skipping Player ID: {player_id}')
        failed_totals.append(player_id)
    except Exception as e: 
        print(f'Trouble getting player {player_id}')
        failed_totals.append(player_id)

    iteration += 1
 
df_totals
df_totals.to_csv('../Datasets/nba_totals.csv')

In [110]:
df = pd.read_csv('../Datasets/nba.csv')
df = df.drop(['Unnamed: 0'], axis= 1)
df.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630173,2020-21,0,1610612748,MIA,21.0,61,4,12.1,2.0,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,1630173,2021-22,0,1610612761,TOR,22.0,73,28,23.6,3.6,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
2,1630173,2022-23,0,1610612761,TOR,23.0,55,12,20.7,3.6,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
3,1630173,2023-24,0,1610612761,TOR,24.0,25,0,17.5,3.1,...,0.571,2.0,3.4,5.4,1.8,0.6,0.5,1.2,1.6,7.7
4,1630173,2023-24,0,1610612752,NYK,24.0,49,18,24.2,3.2,...,0.643,2.9,4.3,7.2,1.1,0.6,1.1,1.1,2.1,7.6


In [109]:
df_totals = pd.read_csv('../Datasets/nba_totals.csv')
df_totals = df_totals.drop('Unnamed: 0', axis = 1)
df_totals = df_totals.loc[:, ['PLAYER_ID', 'GP', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']]
df_totals = df_totals.groupby(['PLAYER_ID'], sort= False).sum()
df_totals['FG_PCT'] = np.round(df_totals['FGM'] / df_totals['FGA'], 3)
df_totals['3_PCT'] = np.round(df_totals['FG3M'] / df_totals['FG3A'], 3)
df_totals['FT_PCT'] = np.round(df_totals['FTM'] / df_totals['FTA'], 3)
df_totals = df_totals.drop(['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA'], axis = 1)
player_names = pd.Series([])

for i in df_totals.loc[:, 'REB':'PTS'].columns:
    df_totals[i] = np.round(df_totals[i] / df_totals['GP'], 1)

for dict in all_players:
    if dict.get('id') in df_totals.index:
        player_names = np.append(player_names, dict.get('full_name'))

df_totals['PLAYER_NAME'] = player_names

df_totals.head()

,GP,REB,AST,STL,BLK,TOV,PTS,FG_PCT,3_PCT,FT_PCT,PLAYER_NAME
PLAYER_ID,,,,,,,,,,,
1630173,394,5.8,1.0,0.6,0.7,1.0,7.6,0.488,0.297,0.608,Precious Achiuwa
203500,769,8.0,1.5,0.8,0.9,1.4,8.8,0.586,0.059,0.535,Steven Adams
1628389,573,8.9,3.6,1.1,0.9,2.2,15.8,0.535,0.317,0.758,Bam Adebayo
1630534,286,2.8,1.2,0.6,0.5,0.8,7.2,0.440,0.336,0.713,Ochai Agbaji
1630583,242,5.3,1.9,0.6,0.6,0.9,9.8,0.458,0.344,0.684,Santi Aldama
